In [6]:
import os
import time
import pyvips
import glob
import numpy as np
import pyvips
from tqdm import tqdm
import re
from PIL import Image
import random
Image.MAX_IMAGE_PIXELS = None

In [7]:
GT_ADDR = '/BrainSeg/ground_truth/'
WSI_ADDR = '/BrainSeg/norm_png/'
TRAIN_ADDR = '/BrainSeg/classify_train_dataset/'
if not os.path.exists(TRAIN_ADDR):
    os.makedirs(TRAIN_ADDR)

In [10]:
ground_truth = sorted(os.listdir(GT_ADDR))
# ground_truth = ground_truth[-5:-4]
ground_truth

['NA4894-02_AB17-24.npy',
 'NA4944-02_AB17-24.npy',
 'NA4967-02_AB17-24.npy',
 'NA4971-02_AB17-24.npy',
 'NA4972-02_AB17-24.npy',
 'NA4992-02_AB17-24.npy',
 'NA4993-02_AB17-24.npy',
 'NA5010-02_AB17-24.npy']

In [9]:
!rm -rf /BrainSeg/ground_truth/.ipynb_checkpoints

In [5]:
gt = np.load(GT_ADDR + ground_truth[0])
gt

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int8)

In [6]:
ground_truth[0].split('.')[0]

'NA4971-02_AB17-24'

In [11]:
# 1: background
# 2: GM
# 4: WM


for truth in ground_truth:
    start = time.time()
    truth_name = truth.split('.')[0]
#     WSI_ADDR = WSI_ADDR + truth_name + '.png'
    WSI = pyvips.Image.new_from_file(WSI_ADDR + truth_name + '.png')
    print(WSI)
    truth_npy = np.load(GT_ADDR + truth)
    
    width = WSI.width
    height = WSI.height
    
    
    step = 512
    width_num = width // step
    height_num = height // step
    print(truth, 'width:', width, 'width_num:', width_num, 'height:', height, 'height_num:', height_num)
    SAVE_GM_DIR = TRAIN_ADDR + 'GM/' + truth_name
    if not os.path.exists(SAVE_GM_DIR):
        os.makedirs(SAVE_GM_DIR) 

    SAVE_WM_DIR = TRAIN_ADDR + 'WM/' + truth_name
    if not os.path.exists(SAVE_WM_DIR):
        os.makedirs(SAVE_WM_DIR)

    SAVE_BG_DIR = TRAIN_ADDR + 'background/' + truth_name
    if not os.path.exists(SAVE_BG_DIR):
        os.makedirs(SAVE_BG_DIR)
    
    NUM_GM = 0
    NUM_WM = 0
    NUM_BG = 0
    for i in range(1, width_num - 1):
        for j in range(1, height_num - 1):
            rd = random.randint(0,128)
            x = i * step + rd 
            y = j * step + rd 
            patch = WSI.extract_area(x-128, y-128, 256, 256)
            index_value = truth_npy[x,y]
            if index_value == 2:
                patch_name = SAVE_GM_DIR + '/' + 'g' + str(NUM_GM) + '.png'
                NUM_GM = NUM_GM + 1
            elif index_value == 4:
                patch_name = SAVE_WM_DIR + '/' + 'w' + str(NUM_WM) + '.png'
                NUM_WM = NUM_WM + 1
            else:
                patch_name = SAVE_BG_DIR + '/' + 'b' + str(NUM_BG) + '.png'
                NUM_BG = NUM_BG + 1
#             patch_name = SAVE_GM_DIR + '/'+ Tile_Name + str(i) + '.png'
    
            patch.write_to_file(patch_name)
    end = time.time()
    print(truth, end - start)

<pyvips.Image 59760x44874 uchar, 3 bands, srgb>
NA4894-02_AB17-24.npy width: 59760 width_num: 116 height: 44874 height_num: 87
NA4894-02_AB17-24.npy 362.16374158859253
<pyvips.Image 47808x36870 uchar, 3 bands, srgb>
NA4944-02_AB17-24.npy width: 47808 width_num: 93 height: 36870 height_num: 72
NA4944-02_AB17-24.npy 192.34869647026062
<pyvips.Image 47808x35729 uchar, 3 bands, srgb>
NA4967-02_AB17-24.npy width: 47808 width_num: 93 height: 35729 height_num: 69
NA4967-02_AB17-24.npy 183.88225984573364
<pyvips.Image 61752x38443 uchar, 3 bands, srgb>
NA4971-02_AB17-24.npy width: 61752 width_num: 120 height: 38443 height_num: 75
NA4971-02_AB17-24.npy 258.7615315914154
<pyvips.Image 47808x39068 uchar, 3 bands, srgb>
NA4972-02_AB17-24.npy width: 47808 width_num: 93 height: 39068 height_num: 76
NA4972-02_AB17-24.npy 195.72688484191895
<pyvips.Image 37848x37641 uchar, 3 bands, srgb>
NA4992-02_AB17-24.npy width: 37848 width_num: 73 height: 37641 height_num: 73
NA4992-02_AB17-24.npy 165.895373106002